In [49]:
%load_ext autoreload
%autoreload 2
%matplotlib inline  

import pandas as pd
import numpy as np

from datetime import datetime, timedelta
import parseIntervalFiles as pif
import parseActivityFiles as paf
#pun intended :)
import consolidateFiles as cf
import datacleaning as cl

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## ! Parse activity files and parse interval files are to be replaced with the corresponding database queries as soon as they are available 

## 0 - Pipeline configuration 
* set the input/output directories, user id and verbose level

In [50]:
verbose = True
    
DATA_PATH = "C:\\Users\\ju\\GDrive\\Projects\\HeRV\\Data\\" 
# DATA_PATH = "/home/ju/GDrive/Projects/HeRV/Data/"
    
RAW_PATH = DATA_PATH + "Raw"
PRE_PATH = DATA_PATH + "PreProcessed"     

## 1 - Extract sessions from raw data (intervals + annotations files)

### 1.1 - List activities with session start/stop datetime
Parses activities files and prints all errors found (activities without start or stop, invalid dates, etc)

In [51]:
sessions = []
for user in range(7):
    print ("------- parsing activity files for user ", user, " -------")
    user_sess = paf.get_user_sessions(user, dirname=RAW_PATH)
    print ("------- parsing interval files for user ", user, " -------")
    user_sess = cf.sessions_add_beats(sessions=user_sess, dirname=RAW_PATH+"\\"+str(user), verbose=False)
    sessions.extend(user_sess)

------- parsing activity files for user  0  -------
reading act170929.csv ... 
reading act170930.csv ... 
reading act171001.csv ... 
reading act171003.csv ... 
reading act171004.csv ... 
reading act171005.csv ... 
orphan start in: ['2017-10-05 12:54:00', 'start', 'eat', 'sitting', '']
orphan start in: ['2017-10-05 13:48:39', 'start', 'rest-active', 'sitting', '']
orphan start in: ['2017-10-05 13:57:37', 'start', 'movement', 'standing', '']
orphan start in: ['2017-10-05 17:55:44', 'start', 'focused-active', 'sitting', '']
orphan start in: ['2017-10-05 21:53:45', 'start', 'rest-active', 'sitting', '']
orphan start in: ['2017-10-05 23:47:45', 'start', 'focused-active', 'sitting', 'sitting on bed, installing and using pandas']
reading act171006.csv ... 
reading act171010.csv ... 
orphan start in: ['2017-10-10 23:23:37', 'start', 'leisure-passive', 'sitting']
reading act171011.csv ... 
orphan stop in: ['2017-10-11 0:04:41', 'stop', '', '', '']
orphan start in: ['2017-10-11 12:30:10', 'start

In [71]:
df = pd.DataFrame(sessions)

### Include column with beats count for ease of use 

In [72]:
df['beatscount'] = df['rr'].apply(len)

In [73]:
df.sample(5)

,activity,duration,notes,posture,rr,start,stop,user,beatscount
349,movement,1320,NaN,stand,"[{'date': 2017-10-06 15:32:01, 'interval': 705...",2017-10-06 15:32:00,2017-10-06 15:54:00,2,1223
369,eat,1560,NaN,sit,"[{'date': 2017-10-08 17:33:01, 'interval': 697...",2017-10-08 17:33:00,2017-10-08 17:59:00,2,1865
26,rest-passive,909,NaN,sitting,"[{'date': 2017-10-03 18:29:03, 'interval': 684...",2017-10-03 18:29:02,2017-10-03 18:44:11,0,1223
187,rest-passive,734,NaN,sit,"[{'date': 2017-11-24 12:37:28, 'interval': 561...",2017-11-24 12:37:27,2017-11-24 12:49:41,0,382
59,household-chores,1000,NaN,sitting,"[{'date': 2017-10-11 18:09:33, 'interval': 657...",2017-10-11 18:09:32,2017-10-11 18:26:12,0,1139


## 2 - Cleaning data

### 2.1 Removing outliers and artifacts

(Note: For now, beats won't be saved to the output file, so, when loading the sessions file and getting
beats from DB again, for example for fragmentation, they will have to be removed again. But this is a reasonably low-cost op)

In [74]:
df['rr'] = df['rr'].apply(cl.clean_rr_series)

In [75]:
df['beatscount_clean'] = df['rr'].apply(len)

In [76]:
df['removed_artifacts'] = df['beatscount'] - df['beatscount_clean']
df['beatscount'] = df['beatscount_clean']
df = df.drop(['beatscount_clean'], axis=1)

In [69]:
df.describe()

,duration,user,beatscount,removed_artifacts
count,453.000000,453.000000,453.000000,453.000000
mean,3144.185430,1.275938,3948.962472,34.432671
std,5745.142626,1.573078,6517.185066,227.407441
min,88.000000,0.000000,105.000000,0.000000
25%,900.000000,0.000000,1097.000000,0.000000
50%,1426.000000,1.000000,1768.000000,1.000000
75%,2460.000000,2.000000,3789.000000,14.000000
max,34360.000000,6.000000,46856.000000,4670.000000


### 2.2 Removing sessions without beats recorded or with too few of them

In [77]:
l = len(df)
df = df[df['beatscount'] > 100]
print(l - len(df), ' sessions out of', l, 'removed for lack of interval data')

44  sessions out of 497 removed for lack of interval data


### 2.3 Removing wrong activities and grouping the ones with too few examples

In [79]:
df.groupby('activity').count()['duration']

activity
eat                 52
exercise-high       17
exercise-low         3
focused-active      95
focused-active       1
focused-passive     43
focused-passive      1
household-chores    28
leisure-active       7
leisure-passive     46
movement            54
not-recorded         4
rest-active         27
rest-passive        22
sleep               53
Name: duration, dtype: int64

In [80]:
# we have too few exercise sessions to classify intensity
df.at[df['activity']=='exercise-low', 'activity'] = 'exercise'
df.at[df['activity']=='exercise-high', 'activity'] = 'exercise'

# also we have too few leisure active sessions
df.at[df['activity']=='leisure-active', 'activity'] = 'leisure'
df.at[df['activity']=='leisure-passive', 'activity'] = 'leisure'

# these were just mispelled
df.at[df['activity']=='focused-passive ', 'activity'] = 'focused-passive'
df.at[df['activity']=='focused-active ', 'activity'] = 'focused-active'

df = df[df['activity']!='not-recorded']

In [81]:
df.groupby('activity').count()['duration']

activity
eat                 52
exercise            20
focused-active      96
focused-passive     44
household-chores    28
leisure             53
movement            54
rest-active         27
rest-passive        22
sleep               53
Name: duration, dtype: int64

## 3 - Aggregating data

In [82]:
dic = df.to_dict(orient='records')
for i in dic:
    i.update(cf.features_from_dic(i['rr']))
print(dic[0]['rmssd'])

239.316870462


In [83]:
df = pd.DataFrame(dic)
df.sample(3)

,activity,beatscount,duration,hf,hfnu,lf,lf_hf,lfnu,mhr,mrri,...,posture,removed_artifacts,rmssd,rr,sdnn,start,stop,total_power,user,vlf
90,focused-active,3253,2738,6473.122730,61.154796,4111.693472,0.635195,38.845204,74.433553,832.804796,...,sitting,11,138.306105,"[{'date': 2017-10-15 21:19:53, 'interval': 100...",154.765788,2017-10-15 21:19:52,2017-10-15 22:05:30,14110.706626,0,3525.890424
78,focused-active,894,650,363.604594,45.094356,442.714925,1.217572,54.905644,81.127647,742.578300,...,sitting,0,31.270485,"[{'date': 2017-10-14 19:11:03, 'interval': 778...",47.637278,2017-10-14 19:11:02,2017-10-14 19:21:52,1567.095039,0,760.775520
192,movement,5967,2881,929.875829,56.050760,729.112967,0.784097,43.949240,124.469577,489.656444,...,stand,25,63.524823,"[{'date': 2018-02-21 14:24:20, 'interval': 483...",68.065614,2018-02-21 14:24:12,2018-02-21 15:12:13,2122.874038,0,463.885242


In [84]:
df['sess_id'] = df.index
df_export = df.drop(['rr'], axis = 1)
df_export.sample(1)

,activity,beatscount,duration,hf,hfnu,lf,lf_hf,lfnu,mhr,mrri,...,posture,removed_artifacts,rmssd,sdnn,start,stop,total_power,user,vlf,sess_id
359,household-chores,5073,3000,142.559188,17.457888,674.029791,4.72807,82.542112,99.747046,604.883895,...,stand,4,17.339615,45.856557,2017-10-17 21:00:00,2017-10-17 21:50:00,1182.822096,2,366.233117,359


In [85]:
df_export.describe()

,beatscount,duration,hf,hfnu,lf,lf_hf,lfnu,mhr,mrri,nn50,pnn50,removed_artifacts,rmssd,sdnn,total_power,user,vlf,sess_id
count,449.000000,449.000000,449.000000,449.000000,449.000000,449.000000,449.000000,449.000000,449.000000,449.000000,449.000000,449.000000,449.000000,449.000000,449.000000,449.000000,449.000000,449.000000
mean,3956.723831,3151.884187,2180.088540,41.939632,1736.148680,1.939087,58.060368,87.244239,731.137163,954.351893,17.545222,34.641425,65.835872,91.594234,5302.848776,1.269488,1386.611556,224.000000
std,6532.976260,5762.267147,4391.992246,17.161628,1910.433850,1.599788,17.161628,18.976790,142.258513,2664.966602,15.576111,228.402474,52.834681,44.369566,6978.924860,1.578590,1248.666812,129.759393
min,105.000000,88.000000,13.635122,8.241489,36.995074,0.172079,14.681505,54.190501,369.893825,0.000000,0.000000,0.000000,5.402758,21.083352,150.076646,0.000000,98.372239,0.000000
25%,1108.000000,900.000000,370.564885,28.873241,674.029791,0.817676,44.984680,74.672025,641.662587,85.000000,5.971151,0.000000,32.724557,62.458526,1752.397329,0.000000,575.914227,112.000000
50%,1779.000000,1430.000000,721.050594,39.793389,1218.595371,1.512980,60.206611,83.213058,733.406699,243.000000,13.684211,1.000000,48.366812,78.552601,3207.787111,1.000000,1028.660179,224.000000
75%,3789.000000,2460.000000,1639.893215,55.015320,1976.091865,2.463415,71.126759,95.145346,817.112689,532.000000,25.096031,14.000000,77.690378,108.602310,5278.123295,2.000000,1718.395697,336.000000
max,46856.000000,34360.000000,49458.460876,85.318495,20551.688970,11.133730,91.758511,164.397688,1144.581425,20895.000000,81.780538,4670.000000,365.184163,269.251749,74465.121914,6.000000,7350.357547,448.000000


In [86]:
df_export.to_excel(PRE_PATH + "\\sessions.xlsx")